### ספריות


In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
import os
import sys
import math
import warnings
from geopy.geocoders import Nominatim
import pyproj

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### העלת משתנים להרצת הקוד


In [2]:
cwd = os.getcwd()

education_folder_path = os.path.dirname(cwd)

sys.path.append(education_folder_path)

### פונקציות גלובליות


In [3]:
from global_functions import remove_spaces_in_columns, up_load_shp, up_load_df, drop_geo, change_cols_names, get_newest_date_file, get_output_folder_location

### הגדרת מיקום הפלט


In [4]:
output_folder_location=get_output_folder_location(r'{}\background_files\output_folder_location.txt'.format(cwd))
out_shp = r'{}\education.shp'.format(output_folder_location)

## העלאת טבלאות


In [5]:
#בתי ספר וגנים מעיריית בית שמש
BShemesh=up_load_df(r'{}\betshemesh_muni\Intermediates'.format(cwd),'BShemesh_final')

In [6]:
#בתי ספר וגנים מעיריית ירושלים
JLM=up_load_df(r'{}\jerusalem_muni\Intermediates'.format(cwd),'JLM_final')

In [7]:
#בתי ספר ממשרד החינוך במרחב
Gschool=up_load_df(r'{}\ministry_of_education\Intermediates'.format(cwd),'Gschool_final')

In [8]:
BShemesh.head()

,ID,x,y,SRC,sector,ele_stu,mid_stu,high_stu
0,114397,199497.00,627750.00,moe_mosdot_coordinates,2,327,100,0
1,174748,199540.00,627386.00,moe_mosdot_coordinates,2,636,225,0
2,238527,198583.00,628208.00,moe_mosdot_coordinates,2,0,39,112
3,273086,199476.00,625922.00,moe_mosdot_coordinates,2,336,97,0
4,721175,198715.00,628209.00,moe_mosdot_coordinates,2,0,57,143


In [9]:
JLM.head()

,ID,x,y,SRC,ele_stu,mid_stu,high_stu,sector
0,21321,216876,629987,jerusalem_muni,230.00,0.00,0.00,3
1,21323,216154,630603,jerusalem_muni,260.25,86.75,0.00,3
2,21326,218166,632109,jerusalem_muni,534.00,0.00,0.00,3
3,21327,220805,629266,jerusalem_muni,331.00,0.00,0.00,3
4,21328,220633,628408,jerusalem_muni,367.00,0.00,0.00,3


In [10]:
Gschool.head()

,ID,x,y,SRC,ele_stu,mid_stu,high_stu,sector
0,110098,190867.00,634787.00,moe_mosdot_coordinates,420.00,0.00,0.00,3
1,110726,194187.00,634711.00,moe_mosdot_coordinates,349.00,0.00,0.00,3
2,110734,197528.00,630138.00,moe_mosdot_coordinates,513.00,0.00,0.00,3
3,111096,214636.00,631692.00,moe_mosdot_coordinates,545.00,0.00,0.00,3
4,111195,211257.00,604302.00,moe_mosdot_coordinates,293.00,0.00,0.00,3


In [11]:
education=pd.concat([BShemesh, JLM, Gschool], ignore_index=True)
education.head()

,ID,x,y,SRC,sector,ele_stu,mid_stu,high_stu
0,114397,199497.00,627750.00,moe_mosdot_coordinates,2,327.00,100.00,0.00
1,174748,199540.00,627386.00,moe_mosdot_coordinates,2,636.00,225.00,0.00
2,238527,198583.00,628208.00,moe_mosdot_coordinates,2,0.00,39.00,112.00
3,273086,199476.00,625922.00,moe_mosdot_coordinates,2,336.00,97.00,0.00
4,721175,198715.00,628209.00,moe_mosdot_coordinates,2,0.00,57.00,143.00


In [12]:
geometry = [Point(xy) for xy in zip(education['x'], education['y'])]
gdf = gpd.GeoDataFrame(education, geometry=geometry, crs='EPSG:2039')

# הגדרת מערכת הקואורדינטות כ-EPSG:2039
# gdf.set_crs(epsg=2039, inplace=True)
gdf.rename(columns={'SRC': 'coordinates_src'}, inplace=True)
gdf = gdf.drop(['x', 'y'], axis=1)

In [13]:
type(gdf['ele_stu'][0])

numpy.float64

In [14]:
# gdf = gdf[
#     (gdf['ele_stu'] != '0') &
#     (gdf['mid_stu'] != '0') &
#     (gdf['high_stu'] != '0')
# ]

gdf = gdf[~((gdf['ele_stu'] == 0) & 
                                  (gdf['mid_stu'] == 0) & 
                                  (gdf['high_stu'] == 0))]

education = education[~((education['ele_stu'] == 0) & 
                                  (education['mid_stu'] == 0) & 
                                  (education['high_stu'] == 0))]


In [15]:
gdf.to_file(out_shp, driver="ESRI Shapefile")
education.to_excel(r'{}\education_final.xlsx'.format(output_folder_location), index=False)

C:\Users\dpere\AppData\Local\Temp\ipykernel_19388\4059256680.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(out_shp, driver="ESRI Shapefile")


In [16]:
education.head()

,ID,x,y,SRC,sector,ele_stu,mid_stu,high_stu
0,114397,199497.00,627750.00,moe_mosdot_coordinates,2,327.00,100.00,0.00
1,174748,199540.00,627386.00,moe_mosdot_coordinates,2,636.00,225.00,0.00
2,238527,198583.00,628208.00,moe_mosdot_coordinates,2,0.00,39.00,112.00
3,273086,199476.00,625922.00,moe_mosdot_coordinates,2,336.00,97.00,0.00
4,721175,198715.00,628209.00,moe_mosdot_coordinates,2,0.00,57.00,143.00


In [17]:
gdf.head()

,ID,coordinates_src,sector,ele_stu,mid_stu,high_stu,geometry
0,114397,moe_mosdot_coordinates,2,327.00,100.00,0.00,POINT (199497.000 627750.000)
1,174748,moe_mosdot_coordinates,2,636.00,225.00,0.00,POINT (199540.000 627386.000)
2,238527,moe_mosdot_coordinates,2,0.00,39.00,112.00,POINT (198583.000 628208.000)
3,273086,moe_mosdot_coordinates,2,336.00,97.00,0.00,POINT (199476.000 625922.000)
4,721175,moe_mosdot_coordinates,2,0.00,57.00,143.00,POINT (198715.000 628209.000)
